# Notebook to find transactions
----
In this notebook we'll process files from a sample data file folder to try and find a transaction.

Need to add:
- some regex to strip symbols
- some strip to take care of any white space
- some more dict variants
- a method to handle the search

We need to:
- Have a process to go through a single report and find that transactions in the other files


In [1]:
import pandas as pd
import numpy as np
import glob
import os
from datetime import timedelta, time, datetime, date
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import warnings
warnings.filterwarnings("ignore")

In [2]:
os.listdir('sample_data')

['.ipynb_checkpoints',
 'Acme Trading Inc_trades_dec_2022.csv',
 'Another Company_trades_dec_2022.csv',
 'Glengarry Glen Ross_trades_dec_2022.csv',
 'Joe Bloggs Investments_trades_dec_2022.csv',
 'Trotters Independent Traders_trades_dec_2022.csv']

In [3]:
# Check sample file
sample_df = pd.read_csv('sample_data/Acme Trading Inc_trades_dec_2022.csv')


In [4]:
sample_df.head()

,timestamp,instrument,quantity,price,trade,counterparty
0,2022-12-01T13:29:59,GB-01234567-05,30000,0.08,sold to,Another Company
1,2022-12-02T18:19:09,GB-01234567-02,50000,0.02,bought from,Another Company
2,2022-12-02T21:05:49,GB-01234567-05,90000,0.09,sold to,Another Company
3,2022-12-02T21:51:12,GB-01234567-02,80000,0.04,sold to,Another Companies
4,2022-12-03T09:07:52,GB-01234567-03,30000,0.01,bought from,Another Company


In [5]:
def check_for_counterparty(company_name, file_name):
    """
    Function to return a DataFrame if the counterparty name 
    has been found.
    """
    
    df = pd.read_csv(file_name)
    counterparties = df.counter_party.unique()
    if company_name in counterparties:
        return df
    else:
        return None

In [6]:
def get_first_dict(df):
    """
    Function to return a dictionary to check that assumes all data was in good shape.
    """
    dictionary = df.groupby(['timestamp', 'instrument', 'quantity',
                   'price', 'counterparty', 'trade'])['timestamp'].count().to_dict()
    
    return dictionary
    

In [7]:
def get_second_dict(df):
    """
    Function to return a dictionary to check that assumes timestamp was wrong.
    
    Here we take the minutes and seconds off of the timestamp.
    """
    
    df['ymd'] = df.timestamp.apply(lambda x: datetime.fromisoformat(x).strftime('%Y-%m-%d'))
    
    dictionary = df.groupby(['ymd', 'instrument', 'quantity',
                   'price', 'counterparty', 'trade'])['timestamp'].count().to_dict()
    
    return dictionary

In [8]:
def check_name_is_in_cohort(df, column):
    """
    Function to check is a name is within a cohort of names contained within
    the df. This is a crude check that a spelling mistake in a name is not 
    preventing a match.
    
    inputs:
    -------
    df (DataFrame): the DataFrame that contains the column
    column (string): the column within the DataFrame that needs to 
        be checked.
        
    outputs:
    --------
    df (DataFrame): the DataFrame with adjusted names in the
        specified column.
    """
    
    output_df = df.copy()
    
    try:
    
        # We check for outliers of names
        unique_names = df[column].value_counts()

        filter_value = 4

        rare_names = unique_names.where(unique_names < filter_value).dropna().index.values.tolist()
        common_names = unique_names.where(unique_names >= filter_value).dropna().index.values.tolist()
        print(common_names)

        if len(rare_names) > 0:
            for name in rare_names:
                # print(f"This is a rare name: {name}")
                # print("----------")
                replacement_name = None
                match_dict = {}

                # Get the matches
                for common_name in common_names:
                    match_dict[common_name] = fuzz.ratio(name, common_name)

                matches = pd.Series(match_dict)
                # print("These are names to match:")
                # print(matches)
                significant_matches = matches.where(matches > 70).dropna()
                best_match = significant_matches.sort_values(ascending=False).index.values.tolist()

                if len(best_match) > 0:            
                    # print("\n")
                    # print("This is the best match")
                    # print(best_match[0])
                    replacement_name = best_match[0]
                    # print("\n")

                output_df[column] = np.where(output_df[column] == name, replacement_name, output_df[column])

        return output_df
    except Exception as error:
        print(f"Couldn't run the name matching function. Error: {error}")
        return df



In [9]:
def get_third_dict(df):
    """
    Function to perform cleaning steps and then produce a dict for searching.
    
    Ensure that the clean df if passed to this function to prevent passing on 
    potential errors from previous steps.
    
    Cols we need to clean:
        'timestamp' - reduce to YMD
        'instrument' - clean the string of whitespace 
        'quantity' - ensure it is a float value, no characters
        'price' - ensure it is a float value, no characters 
        'trade' - ensure that it conforms with expected types
        'counterparty' - clean the string of whitespace, 
            correct low risk typos
    """
    
    # Reduce to YMD
    df['ymd'] = df.timestamp.apply(lambda x: datetime.fromisoformat(x).strftime('%Y-%m-%d'))
    
    # Clean whitespace from the string cols
    string_cols = ['instrument', 'trade', 'counterparty']
    for col in string_cols:
        df[col] = df[col].str.strip()
        
    float_cols = ['quantity', 'price']
    for col in float_cols:
        df[col] = df[col].apply(lambda x: float(re.sub('[^0-9]+', "", str(x))))
        
    # Use Levenshtein distance to match some string values if wrong.
    df = check_name_is_in_cohort(df, "counterparty")
    
    dictionary = df.groupby(['ymd', 'instrument', 'quantity',
               'price', 'counterparty', 'trade'])['timestamp'].count().to_dict()
    
    return dictionary


In [10]:
third_dict = get_third_dict(sample_df)

['Another Company', 'Trotters Independent Traders', 'Glengarry Glen Ross', 'Joe Bloggs Investments']


# Match transactions to files in a folder
----

We need a function that will run through a DataFrame and find matching transactions in files within a folder.

It needs to:
- Perform a quick name check first
- Perform more data cleaning if required
- We want to limit the number of times that we read in a dataset to reduce memory burden

In [27]:
def find_transaction_files(df, starting_file, folder_path, company_name):
    """
    Function to append a column onto the original df that would contain the file 
    name that contained a matching transaction.
    
    inputs:
    -------
    df (DataFrame): the Dataframe that contains the transaction information to find.
    starting_file (string): the name of the starting file that will not be searched.
    folder_path (string): the path to the folder that contains the files to be searched.
    
    outputs:
    --------
    new_df (DataFrame): a DataFrame that is a copy of the input DataFrame with an additional
        column that contains a filename as a string.
    search_dict (Dictionary): nested dictionaries for checking and future use
    matches_df (DataFrame): a DataFrame the contains the file matches.
    """
    
    # Get the unique counterparty names
    df_counter_party_names = df['counterparty'].unique()
    
    files_to_search = os.listdir(folder_path)
    
    if '.ipynb_checkpoints' in files_to_search:
        files_to_search.remove('.ipynb_checkpoints')
    
    search_dicts = {}
    
    try:
        # Read in the data from the file, if there is no match, move onto next file
        for input_file in files_to_search:
            # print(input_file)
            if input_file != starting_file:

                # Read in the file.
                this_file = pd.read_csv(f"{folder_path}/{input_file}")

                this_file_unique_names = this_file['counterparty'].unique()

                overlap = len(set(df_counter_party_names).intersection(set(this_file_unique_names)))

                # If overlap is zero there are no potential matches based on 
                # the information available.
                if overlap == 0:
                    return

                # Collect dicts
                dict_collection = {}
                first_dict = get_first_dict(this_file),
                second_dict = get_second_dict(this_file),
                third_dict = get_third_dict(this_file)
                
                dict_collection["first_dict"] = first_dict
                dict_collection["second_dict"] = second_dict
                dict_collection["third_dict"] = third_dict

                # Insert dicts into search_dicts
                search_dicts[input_file] = dict_collection
    except Exception as error:
        print(f"Couldn't process the file. Error: {error}")
        
    trade_inversion = {
        "bought from": "sold to",
        "sold to": "bought from"
    }
    
    
    df['first_dict_lookup'] = df.apply(lambda x: (x['timestamp'], x['instrument'], x['quantity'],
                                                 x['price']/100, company_name, trade_inversion[x['trade']]), axis=1)
    

    # Compare the dicts
    # print(search_dicts.keys())
    
    matches_df = pd.DataFrame(df.first_dict_lookup, columns=['first_dict_lookup'])
    print(matches_df.head())
    
    for key in search_dicts.keys():
        file_dict = search_dicts[key]
        first_dict = file_dict['first_dict'][0]
        matches_df[key] = matches_df.first_dict_lookup.map(first_dict)
        
        
        
    return df, search_dicts, matches_df


In [28]:
new_df, search_dicts, matches_df = find_transaction_files(sample_df, "Acme Trading Inc_trades_dec_2022.csv",
                                                          'sample_data', company_name="Acme Trading Inc")

['Acme Trading Inc', 'Joe Bloggs Investments', 'Trotters Independent Traders', 'Glengarry Glen Ross']
['Trotters Independent Traders', 'Joe Bloggs Investments', 'Acme Trading Inc', 'Another Company']
['Another Company', 'Trotters Independent Traders', 'Glengarry Glen Ross', 'Acme Trading Inc']
['Glengarry Glen Ross', 'Another Company', 'Joe Bloggs Investments', 'Acme Trading Inc']
                                   first_dict_lookup
0  (2022-12-01T13:29:59, GB-01234567-05, 30000.0,...
1  (2022-12-02T18:19:09, GB-01234567-02, 50000.0,...
2  (2022-12-02T21:05:49, GB-01234567-05, 90000.0,...
3  (2022-12-02T21:51:12, GB-01234567-02, 80000.0,...
4  (2022-12-03T09:07:52, GB-01234567-03, 30000.0,...


In [29]:
matches_df

,first_dict_lookup,Another Company_trades_dec_2022.csv,Glengarry Glen Ross_trades_dec_2022.csv,Joe Bloggs Investments_trades_dec_2022.csv,Trotters Independent Traders_trades_dec_2022.csv
0,"(2022-12-01T13:29:59, GB-01234567-05, 30000.0,...",1.0,NaN,NaN,NaN
1,"(2022-12-02T18:19:09, GB-01234567-02, 50000.0,...",1.0,NaN,NaN,NaN
2,"(2022-12-02T21:05:49, GB-01234567-05, 90000.0,...",1.0,NaN,NaN,NaN
3,"(2022-12-02T21:51:12, GB-01234567-02, 80000.0,...",1.0,NaN,NaN,NaN
4,"(2022-12-03T09:07:52, GB-01234567-03, 30000.0,...",1.0,NaN,NaN,NaN
5,"(2022-12-03T15:08:28, GB-01234567-05, 60000.0,...",NaN,NaN,NaN,1.0
6,"(2022-12-04T04:46:11, GB-01234567-04, 20000.0,...",1.0,NaN,NaN,NaN
7,"(2022-12-04T11:32:05, GB-01234567-02, 80000.0,...",1.0,NaN,NaN,NaN
8,"(2022-12-04T19:29:36, GB-01234567-05, 10000.0,...",NaN,NaN,1.0,NaN
9,"(2022-12-06T10:27:00, GB-01234567-01, 60000.0,...",NaN,NaN,NaN,1.0
